<a href="https://colab.research.google.com/github/JooyoungO/machineLearning/blob/main/05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**정형 데이터 vs 비정형 데이터**

정형 데이터 : csv 파일처럼 구조를 가지고 있는 데이터

비정형 데이터 : 텍스트 데이터, 사진, 디지털 음악 등 구조를 가지고 있지 않은 데이터 (6장)

지금까지 배운 머신러닝 알고리즘들은 정형 데이터에 적합

  * k - 최근접 이웃, 선형 회귀, 릿지, 라쏘 ...

# **앙상블 학습**

정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘

대다수의 앙상블 학습 알고리즘은 결정 트리를 기반으로 함

# **랜덤 포래스트**

앙상블 학습 중 가장 유명하고 안정적인 성능을 제공

여러개의 결정 트리를 랜덤하게 만들고 각 결정 트리의 예측을 사용해 최종 예측을 출력
어떻게 결정 트리를 랜덤하게 만들것인가 핵심

1. 훈련하기 위한 데이터를 랜덤하게 만듦

  * 입력한 훈련 데이터에서 랜덤하게 샘플을 추출함

  * 중복된 샘플을 추출할 수 있음

  * 이렇게 만들어진 샘플을 부트스트랩 샘플 이라 함

  * 기본적으로 훈련 세트의 크기와 동일하게 설정됨

  ex. 1000개의 샘플로 구성된 훈련 세트가 주어지면 그 훈련 세트로부터 랜덤하게 1000개의 샘플을 중복이 가능하도록 선택

2. 부트스트랩 심플로 결정 트리를 훈련

  * 각 노드를 분할 할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음

  * RandomForestClassfier 분류 모델은 기본적으로 전체 특성 개수의 제곱근만큼의 특성으로 선택

  * RandomForestClassfier 회귀 모델은 전체 특성을 사용

3. 정해진 수 만큼 위 과정을 반복

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 훈련

분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 결정

회귀일 때는 각 트리의 예측을 평균함

랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음

랜덤 포레스트는 누락된 값이 있어도 처리가 가능하다는 장점이 있음

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
# 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


oob_score

랜덤하게 부트스트랩 샘플을 선택하기 때문에 사용할 수 있는 RandomForestClassifier

# **엑스트라 트리**

랜덤 포레스트와 비슷하게 동작

  * 부트스트랩 샘플을 사용하지 않고, 각 결정 트리를 만들 때 전체 훈련 세트를 사용

  * 노드를 분할할 때 가장 좋은 분할을 찾이 않고 랜덤으로 분할함

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# **그레이디언트 부스팅**

**Bagging vs Boosting**

위 두 방법은 베깅 방법임

여러 개의 모델을 랜덤하게 만들고 각 ㅁ델의 결과를 종합하여 결과를 출력

아래 방법들은 부스팅 방식임

모델을 순차적으로 여러개 만들되 이전 모델을 보완한 새로운 모델을 만들고 최종적으로 생성된 모든 모델을 하나로 합침

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식

사이킷런에서는 기본적으로 깊이가 3인 결정 트리 100개를 사용

깊이가 얕은 결정 트리를 사용하므로 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음

---

그레이디언트란 이름이 붙은 이유는 경사 하강법을 사용하여 트리를 앙상블에 추가하기 때문


분류에서는 로지스틱 손실 함수를 사용

회귀애서는 평균 제곱 오차 함수를 사용

---

경사 하강법이 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정이라면, 그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동함



경사 하강법에서 손실 함수의 낮은 곳으로 천천히 조금씩 이동해야하는 것처럼 그레이디언트 부스팅도 마찬가지이므로 깊이가 얕은 트리를 사용함

학습률 매개변수로 속도를 조절함

예측에 사용될 데이터가 많은 사용함

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강함

학습률을 증가시ㅣ고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음

학습률이 0.1이 기본값

트리의 개수(n_estimator_)는 100이 기본값

In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# **히스토그램 기반 그레이디언트 부스팅**

정형 데이터를 다루는 머신러닝 알고리즘 중에서 가장 인기가 높은 알고리즘

입력 특성을 256개의 구간으로 나누므로 노드를 분할할 때 최적의 분할을 매두 빠르게 찾을 수 있음

256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용함

즉, 어떤 샘플에 누락된 값이 있다면 떼어놓은 구간의 값을 대신 사용함

따라서 입력에 누락된 특성이 있더라도 이름 따로 전처리할 필요가 없음

히스토그램 :  데이터를 일정한 구간으로 쪼개서 막대그래프로 표현

입력 데이터의 각 특성을 256개의 구간으로 나누어 256개의 값으로 변환

max_iter : 부스팅 반복 횟수, 성능을 높이려면 이 매개변수를 조절

In [10]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923